In [ ]:

#!jupyter serverextension enable voila —sys-prefix

In [ ]:
from fastai.text.all import *
import pandas as pd

from textblob import TextBlob
from wordcloud import WordCloud
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets import *

In [ ]:
'''learn_inf = load_learner("/content/drive/My Drive/data/sentiment analysis/export (1).pkl")'''
learn_inf = load_learner("https://ml-sentiment-model.s3.eu-west-2.amazonaws.com/export_1.pkl")

'learn_inf = load_learner("export_1.pkl")'

In [ ]:
import GetOldTweets3 as got
import pandas as pd


user_name = "@metpoliceuk"
q='to:{}'.format(user_name)

In [ ]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch(q)\
                                           .setSince("2020-08-25")\
                                           .setUntil("2020-08-26")
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

df = pd.DataFrame(tweets)

def get_text(tweet):
  return tweet.text

def get_username(tweet):
  return tweet.username

df["username"] = df[0].apply(get_username)
df["text"] = df[0].apply(get_text)

In [ ]:
bios_df = df.copy()
bios = bios_df.dropna(axis=0, subset=["username"])
bios = bios.drop_duplicates(subset=["username"])

In [ ]:
import tweepy
import logging
from tweepy import API

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text

def get_bio(user):
  list_of_bios = pd.DataFrame(columns=["Users","Bios"])
  i = 100
  while i < len(user):
    users = user[i-100:i]
    test = api.lookup_users(screen_names =users)
    for person in test:
      list_of_bios = list_of_bios.append({'Users' : person.screen_name , 'Bios' : person.description}, ignore_index=True)
    i = i + 100
  users = user[i-100:len(user)]
  test = api.lookup_users(screen_names =users)
  for person in test:
      list_of_bios = list_of_bios.append({'Users' : person.screen_name , 'Bios' : person.description}, ignore_index=True)
  return list_of_bios


def get_label(row):
  return row[0]

def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text


list_of_new_bios = get_bio(df["username"].tolist())

In [ ]:
def run_process(user_name, date_from, date_to):

  q='to:{}'.format(user_name)

  tweetCriteria = got.manager.TweetCriteria().setQuerySearch(q)\
                                            .setSince(date_from)\
                                            .setUntil(date_to)
  tweets = got.manager.TweetManager.getTweets(tweetCriteria)

  df = pd.DataFrame(tweets)

  def get_text(tweet):
    return tweet.text

  def get_username(tweet):
    return tweet.username

  df["username"] = df[0].apply(get_username)
  df["text"] = df[0].apply(get_text)

  bios_df = df.copy()
  bios = bios_df.dropna(axis=0, subset=["username"])
  bios = bios.drop_duplicates(subset=["username"])

  auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
  auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

  api = tweepy.API(auth)

  list_of_new_bios = get_bio(df["username"].tolist())

  list_of_new_bios["RW_prediction"] = list_of_new_bios["Bios"].apply(learn_inf.predict)

  list_of_new_bios["RW_prediction"] = list_of_new_bios["RW_prediction"].apply(get_label)

  list_of_new_bios = list_of_new_bios.rename({"Users":"username"}, axis=1)

  new_df = df.merge(list_of_new_bios, on=["username"])



  # Clean the tweets
  new_df['clean_text'] = new_df['text'].apply(cleanTxt)

    # Create two new columns 'Subjectivity' & 'Polarity'
  new_df['Subjectivity'] = new_df['clean_text'].apply(getSubjectivity)
  new_df['Polarity'] = new_df['clean_text'].apply(getPolarity)

  right_wing_mask = new_df["RW_prediction"] == "True"
  right_wing = new_df[right_wing_mask]

  not_right_wing_mask = new_df["RW_prediction"] == "False"
  not_right_wing = new_df[not_right_wing_mask]

  fig2 = px.scatter(new_df, x="Subjectivity", y="Polarity", color="RW_prediction", hover_data=["Bios"])
  fig2.update_xaxes(range=[-0.1,1.1])
  fig2.update_yaxes(range=[-1.1, 1.1])

  grouped_by_RW = new_df.groupby("RW_prediction").agg(
    count= ("text", len),
    Subjectivity = ("Subjectivity", np.mean),
    Polarity = ("Polarity", np.mean))
  
  fig1 = px.bar(grouped_by_RW, y="count", color=grouped_by_RW.index)

  final = round(grouped_by_RW["count"].iloc[1] / grouped_by_RW["count"].iloc[0],3) * 100

  return grouped_by_RW, fig1, fig2, final

# Right Wing Follower Test

This tool is designed to check how many explicitly right wing users interact with a twitter account on a certain day, and how they differ to the rest of the interactions.

In [ ]:
username_select =widgets.Text(
    value='@metpoliceuk',
    placeholder='Type something',
    description='Username:',
    disabled=False
)

date_from_widget =widgets.Text(
    value='2020-08-27',
    placeholder='Type something',
    description='Date From:',
    disabled=False
)

date_to_widget =widgets.Text(
    value='2020-08-28',
    placeholder='Type something',
    description='Date To:',
    disabled=False
)

def show_graph2(change):
  with out_df:
    print("analysing..")
  results = run_process(username_select.value, date_from_widget.value, date_to_widget.value)
  out_df.clear_output()
  out_analysis.clear_output()
  out_graph1.clear_output()
  out_graph2.clear_output()
  with out_df:
    print(str(results[3]) + "% RW interactions")
  with out_analysis:
    print(results[0])
  with out_graph1:
    results[1].show()
  with out_graph2:
    results[2].show()
  
analyse_button = widgets.Button(description='Analyse')

analyse_button.on_click(show_graph2)

out_df = widgets.Output(layout={'border': '1px solid black'})
out_analysis = widgets.Output(layout={'border': '1px solid black'})
out_graph1 = widgets.Output(layout={'border': '1px solid black'})
out_graph2 = widgets.Output(layout={'border': '1px solid black'})


In [ ]:
VBox([widgets.Label('Tweet Monitor'),
      username_select, date_from_widget, date_to_widget, analyse_button, out_df, out_analysis, out_graph1, out_graph2])

In [ ]:

def run_test(change):
  input = str(test_box.value)
  results = learn_inf.predict(input)
  test_out.clear_output()
  with test_out:
    print("Prediction for " + input + " is... "+results[0])

test_button = widgets.Button(description='Test')

test_button.on_click(run_test)


In [ ]:
test_out = widgets.Output(layout={'border': '1px solid black'})


test_box =widgets.Text(
    value='Enter text test here',
    placeholder='Type something',
    description='String:',
    disabled=False
)

In [ ]:
VBox([widgets.Label('Output Test'),
      test_box, test_button, test_out])